In [2]:
import keras
from keras.datasets import imdb
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
import json
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

In [3]:
imdb_path = "/media/roodram/72F8C754F8C7156F/ACM/Project/DataSets/imdb.npz"
(x_train, y_train), (x_test, y_test)= imdb.load_data(path = imdb_path, maxlen=2000, num_words=50000)

In [4]:
x = np.concatenate((x1,x2), axis=0)
y = np.concatenate((y1,y2), axis=0)
print(x.shape)
print(y.shape)


(49996,)
(49996,)


In [5]:
num_words=0
maxlen=0
for i in range(x.shape[0]):
    #print(i)
    l = len(x[i])
    maxlen = max(maxlen, l)
    num_words = max(num_words, max(x[i]))
    
print(num_words)
print(maxlen)

49999
1854


In [6]:
print(type(x1[0]))
maxlen=2000
num_words=50000

<class 'list'>


In [20]:
x = sequence.pad_sequences(x, maxlen=maxlen)
print(x_train.shape)
print(y_train.shape)
print(len(x[0]))

(39996, 2000)
(39996,)
2000


In [32]:
from keras.models import Model
from keras.layers import Dense, Embedding, RNN, Input, Dropout, LSTM, Activation, Conv1D, MaxPooling1D, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam

In [33]:
def CNN_Model_Keras(maxlen, num_words):
    inputs = Input((maxlen,), dtype=np.float32)
    X = Embedding(num_words, 64, input_length=maxlen)(inputs)
    X = Conv1D(64, 8, padding='valid', activation='tanh')(X)
    X = BatchNormalization(axis=2)(X)
    X = MaxPooling1D(3)(X)
    X = Dropout(0.25)(X)
    X = LSTM(64)(X)
    X = Dropout(0.25)(X)
    X = Dense(1, activation='sigmoid')(X)
    model = Model(inputs=inputs, outputs=X)
    return model

In [34]:
model = CNN_Model_Keras(maxlen, num_words)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 2000)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 2000, 64)          3200000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1993, 64)          32832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1993, 64)          256       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 664, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 664, 64)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
__________

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
model.fit(x_train, y_train, epochs = 1, batch_size = 15, shuffle=True)

Epoch 1/1
39996/39996 [==============================] - 1822s 46ms/step - loss: 0.1175 - acc: 0.9600


In [25]:
from keras.models import model_from_json

In [41]:
model_file = 'Models/imdb_cnn_e2.json'
json_model = model.to_json()
with open(model_file, 'w') as file:
    file.write(json_model)
    
weights_file = 'Models/imdb_cnn_e2.h5'
model.save_weights(weights_file)

In [43]:
loss, acc = model.evaluate(x_test, y_test)
print(loss)
print(acc)

10000/10000 [==============================] - 90s 9ms/step
0.31315985962748527
0.8914
